## About
This notebook contains simple, toy examples to help you get started with FairMLHealth tool usage. This same content is mirrored in the repository's main [README](../README.md).

## Example Setup

In [1]:
from fairmlhealth import report as fhrp, measure, stat_utils

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier


In [2]:
# First we'll create a semi-randomized dataframe with specific columns for our attributes of interest
np.random.seed(506)
N = 240
X = pd.DataFrame({'col1': np.random.randint(1, 4, N), 
                  'col2': np.random.randint(1, 75, N),
                  'col3': np.random.randint(0, 2, N),
                  'gender': [0, 1]*int(N/2), 
                  'ethnicity': [1, 1, 0, 0]*int(N/4),
                  'other': [1, 0, 1, 0, 1, 1, 0, 1]*int(N/8)
                 })

# Next we'll create a randomized target value
y = pd.Series(np.random.randint(0, 2, N), name='y')

# Third, we'll split the data and use it to train two generic models
splits = train_test_split(X, y, stratify=y, test_size=0.5, random_state=60)
X_train, X_test, y_train, y_test = splits

model_1 = BernoulliNB().fit(X_train, y_train)
model_2 = DecisionTreeClassifier().fit(X_train, y_train)



In [4]:
display(X.head(), y.head())

,col1,col2,col3,gender,ethnicity,other
0,1,15,0,0,1,1
1,3,51,1,1,1,0
2,1,30,1,0,0,1
3,2,28,1,1,0,0
4,1,72,0,0,1,1


0    0
1    0
2    0
3    1
4    1
Name: y, dtype: int64

## Model Measurement
The primary feature of this library is the model comparison tool. The current version supports assessment of binary prediction models through use of the measure_models and compare_models functions.

Measure_model is designed to generate a report of multiple fairness metrics for a single model. Here it is shown wrapped in a "flag" function to emphasize values that are outside of the "fair" range. 

In [5]:
# Generate a pandas dataframe of measures
fhrp.measure_model(X_test, y_test, X_test['gender'], model_1, flag_oor=True)

## Evaluation

FairMLHealth now also includes stratified table features to aid in identifying the source of unfairness or other bias: data tables, performance tables, and bias tables. Note that these stratified tables can evaluate multiple features at once, and that there are two options for identifying which features to assess.

Note that the flag tool has not yet been updated to work with stratified tables.

### Significance Testing

It's recommended to test for the statistical significance of discrepancies in the distribution of results. This is particularly true for attributes with skewed distributions, for which small sample sizes for less common labels may affect fairness measures. However it is still generally recommended to validate whether the test sample for which fairness measures are generated is reflective of the full dataset.

FairMLHealth comes with a bootstrapping utility and supporting funcitons that can be used in some statistical testing. While the selection of proper statistical tests is beyond the scope of this notebooks, two examples using the bootstrap_significance tool with built-in test functions are shown below.

In [6]:
model_1_preds = pd.Series(model_1.predict(X_test))

In [7]:
# Example Significance Test Results Applying Kruskal-Wallis to Predictions
isMale = X_test.reset_index(drop=True)['gender'].eq(1)
reject_h0 = stat_utils.bootstrap_significance(alpha=0.05,
                                              func=stat_utils.kruskal_pval, 
                                              dist_a=model_1_preds.loc[isMale], 
                                              dist_b=model_1_preds.loc[~isMale])
print("Is it likely that the difference in the mean y value is related to gender?\n", reject_h0)

Is it likely that the difference in the mean y value is related to gender?
 True


In [8]:
# Example Significance Test Results Applying Chi-Square to the Distribution of Prediction Successes/Failures
model_1_results = stat_utils.binary_result_labels(y_test, model_1_preds)
reject_h0 = stat_utils.bootstrap_significance(alpha=0.05,
                                              func=stat_utils.chisquare_pval, 
                                              group=X_test['gender'], 
                                              values=model_1_results)
print("Can we reject the hypothesis that prediction results are from the same", 
      "distribution for each gender?\n", reject_h0)

Can we reject the hypothesis that prediction results are from the same distribution for each gender?
 False


### Stratified Data Tables

The data table is shown below with each of the two data argument options. It evaluates basic statistics specific to each feature-value, in addition to relative statistics for the target value. Since the table can be used to evaluate many features at once, it can be a useful option for identifying patterns of bias either alone or in concert with other (e.g., visual methods).

In [9]:
# Arguments Option 1: pass full set of data, subsetting with *features* argument
measure.data(X_test, y_test, features=['gender', 'other', 'col1'])

,Feature Name,Feature Value,Obs.,Entropy,Mean y,Median y,Missing Values,Std. Dev. y,Value Prevalence
0,ALL FEATURES,ALL VALUES,120,NaN,0.5000,0.5,0,0.5021,1.0000
1,gender,0,58,0.9992,0.4310,0.0,0,0.4995,0.4833
2,gender,1,62,0.9992,0.5645,1.0,0,0.4999,0.5167
3,other,0,43,0.9413,0.5581,1.0,0,0.5025,0.3583
4,other,1,77,0.9413,0.4675,0.0,0,0.5022,0.6417
5,col1,1,41,1.5841,0.5366,1.0,0,0.5049,0.3417
6,col1,2,41,1.5841,0.5366,1.0,0,0.5049,0.3417
7,col1,3,38,1.5841,0.4211,0.0,0,0.5004,0.3167


In [10]:
# Multiple targets can also be passed through the data table 
measure.data(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'col3'])

,Feature Name,Feature Value,Obs.,Entropy,Mean col2,Mean col3,Median col2,Median col3,Missing Values,Std. Dev. col2,Std. Dev. col3,Value Prevalence
0,ALL FEATURES,ALL VALUES,120,NaN,36.3417,0.4917,33.5,0.0,0,21.0779,0.5020,1.0000
1,gender,0,58,0.9992,38.9138,0.4828,38.5,0.0,0,21.5482,0.5041,0.4833
2,gender,1,62,0.9992,33.9355,0.5000,30.0,0.5,0,20.5098,0.5041,0.5167
3,col1,1,41,1.5841,39.8293,0.5122,40.0,1.0,0,21.8620,0.5061,0.3417
4,col1,2,41,1.5841,31.3659,0.4634,25.0,0.0,0,18.4415,0.5049,0.3417
5,col1,3,38,1.5841,37.9474,0.5000,35.5,0.5,0,22.3824,0.5067,0.3167


In [11]:
# The "ALL FEATURES" overview column can be turned off via the add_overview argument
measure.data(X_test[['gender']], y_test, add_overview=False)

,Feature Name,Feature Value,Obs.,Entropy,Mean y,Median y,Missing Values,Std. Dev. y,Value Prevalence
0,gender,0,58,0.9992,0.4310,0.0,0,0.4995,0.4833
1,gender,1,62,0.9992,0.5645,1.0,0,0.4999,0.5167


In [12]:
# Multiple targets can also be passed through the data table 
measure.data(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'other'])

,Feature Name,Feature Value,Obs.,Entropy,Mean col2,Mean other,Median col2,Median other,Missing Values,Std. Dev. col2,Std. Dev. other,Value Prevalence
0,ALL FEATURES,ALL VALUES,120,NaN,36.3417,0.6417,33.5,1.0,0,21.0779,0.4815,1.0000
1,gender,0,58,0.9992,38.9138,0.7759,38.5,1.0,0,21.5482,0.4207,0.4833
2,gender,1,62,0.9992,33.9355,0.5161,30.0,1.0,0,20.5098,0.5038,0.5167
3,col1,1,41,1.5841,39.8293,0.6585,40.0,1.0,0,21.8620,0.4801,0.3417
4,col1,2,41,1.5841,31.3659,0.6098,25.0,1.0,0,18.4415,0.4939,0.3417
5,col1,3,38,1.5841,37.9474,0.6579,35.5,1.0,0,22.3824,0.4808,0.3167


### Stratified Performance Tables

The stratified performance table contains model performance measures specific to each feature-value subset. If prediction probabilities are available to the model, additional ROC AUC and PR AUC values will be included.

In [13]:
measure.performance(X_test[['gender']], y_test, model_1.predict(X_test), add_overview=False)

,Feature Name,Feature Value,Obs.,Accuracy,F1-Score,FPR,Mean __y_pred,Mean __y_true,Precision,TPR
0,gender,0,58.0,0.5517,0.3158,0.2121,0.2241,0.4310,0.4615,0.24
1,gender,1,62.0,0.5806,0.6829,0.7037,0.7581,0.5645,0.5957,0.80


In [14]:
measure.performance(X_test[['gender']], 
                    y_true=y_test, 
                    y_pred=model_1.predict(X_test), 
                    y_prob=model_1.predict_proba(X_test)[:,1])

,Feature Name,Feature Value,Obs.,Accuracy,F1-Score,FPR,Mean __y_pred,Mean __y_true,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,120.0,0.5667,0.5667,0.4333,0.5000,0.5000,NaN,0.5667,0.6154,0.5667
1,gender,0,58.0,0.5517,0.3158,0.2121,0.2241,0.4310,NaN,0.4615,0.5473,0.2400
2,gender,1,62.0,0.5806,0.6829,0.7037,0.7581,0.5645,NaN,0.5957,0.6466,0.8000


### Stratified Bias Fairness Tables

The stratified bias table contains model bias measures specific to each feature-value subset. Inspired by common measures of fairness, the tool assumes each feature-value as the "privileged" group relative to all other possible values for the feature. For example, row 3 in the table below displaying measures of "col1" value of "2" where 2 is considered to be the privileged group and all other values (1, 2, 45, and 50) are considered unprivileged.

To simplify the table, fairness measures have been reduced to their component parts. For example, measures of Equalized Odds can be determined by combining the True Positive Rate (TPR) Ratios & Differences with False Positive Rate (FPR) Ratios & Differences.

See also: [Fairness Quick References](../docs/Fairness_Quick_References.pdf) and the [Tutorial for Evaluating Fairness in Binary Classification](./Tutorial-EvaluatingFairnessInBinaryClassification.ipynb)

In [15]:
measure.bias(X_test[['gender', 'col3']], y_test, model_1.predict(X_test), flag_oor=True)

,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,gender,0,0.0342,1.0666,0.4916,3.3175,0.1342,1.2908,0.5339,3.3821,0.5600,3.3333
1,gender,1,-0.0342,0.9376,-0.4916,0.3014,-0.1342,0.7747,-0.5339,0.2957,-0.5600,0.3000
2,col3,0,0.1862,1.3901,-0.2768,0.5079,0.2492,1.5481,-0.0834,0.8459,0.0957,1.1849
3,col3,1,-0.1862,0.7194,0.2768,1.9688,-0.2492,0.6459,0.0834,1.1821,-0.0957,0.8439


## Comparing Results for Multiple Models

The compare_models feature can be used to generate side-by-side fairness comparisons of multiple models. Model performance metrics such as accuracy and precision are also provided to facilitate comparison.   

Below is an example output comparing the two example models defined above. Missing values have been added for metrics requiring prediction probabilities, which the second model does not have (note the warning below).

In [16]:
# Pass the data and models to the compare models function, as above
fhrp.compare_models(X_test, y_test, X_test['gender'], 
                    {'model 1':model_1, 'model 2':model_2},
                    flag_oor=True)

The compare_models function can also be used to measure two different protected attributes. Protected attributes are measured separately and cannot yet be combined together with this tool.

In [17]:
fhrp.compare_models(X_test, y_test, 
                    [X_test['gender'], X_test['ethnicity']], 
                    {'gender':model_1, 'ethnicity':model_1},
                    flag_oor=True)


## Analysis by Cohort

In [18]:
measure.bias(X_test['col3'], y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=X_test['gender'])

,gender,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,0,col3,0,0.1000,1.2143,-0.1000,0.6250,0.0714,1.1667,-0.0190,0.9184,0.1000,1.5000
1,0,col3,1,-0.1000,0.8235,0.1000,1.6000,-0.0714,0.8571,0.0190,1.0889,-0.1000,0.6667
2,1,col3,0,0.1141,1.2209,-0.3235,0.6071,0.3004,1.6508,-0.1613,0.8077,-0.0952,0.8889
3,1,col3,1,-0.1141,0.8191,0.3235,1.6471,-0.3004,0.6058,0.1613,1.2381,0.0952,1.1250


In [23]:
measure.bias(X_test[['gender']], y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=X_test[['col3', 'other']])

~/repos/fairMLHealth/fairmlhealth/__utils.py:56: UserWarning: Possible error in column(s) ['gender']. Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/__utils.py:56: UserWarning: Possible error in column(s) ['gender']. Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,col3,other,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,0,0,gender,0,0.0000,1.0000,0.0000,1.0000,0.0451,1.1053,0.0000,1.0000,0.0000,1.0000
1,0,0,gender,1,0.0000,1.0000,0.0000,1.0000,-0.0451,0.9048,0.0000,1.0000,0.0000,1.0000
2,0,1,gender,0,0.0143,1.0286,0.5714,0.0000,0.4286,0.0000,0.5833,0.0000,0.6000,0.0000
3,0,1,gender,1,-0.0143,0.9722,-0.5714,0.0000,-0.4286,0.0000,-0.5833,0.0000,-0.6000,0.0000
4,1,0,gender,0,0.0000,1.0000,0.0000,1.0000,0.3182,1.6364,0.0000,1.0000,0.0000,1.0000
5,1,0,gender,1,0.0000,1.0000,0.0000,1.0000,-0.3182,0.6111,0.0000,1.0000,0.0000,1.0000
6,1,1,gender,0,0.1042,1.2083,0.3750,0.0000,0.7000,0.0000,0.5000,0.0000,0.5833,0.0000
7,1,1,gender,1,-0.1042,0.8276,-0.3750,0.0000,-0.7000,0.0000,-0.5000,0.0000,-0.5833,0.0000
